In [1]:
import adal
from msrestazure.azure_active_directory import AADTokenCredentials
from dotenv import load_dotenv, find_dotenv
import os
load_dotenv(find_dotenv()) 

True

In [11]:
os.getenv("AZURE_SUBSCRIPTION")

In [12]:
import requests
 
# Parameters need for API
subscription = os.getenv("AZURE_SUBSCRIPTION_ID")
tenant = os.getenv("AZURE_TENANT_ID")
client_id = os.getenv("AZURE_CLIENT_ID")
client_secret = os.getenv("AZURE_CLIENT_SECRET")
offer = 'MS-AZR-0003P'
currency = 'USD'
locale = 'en-US'
region = 'US'

In [13]:
def authenticate_client_key(tenant, client_id, client_secret):
    """
    Authenticate using service principal w/ key.
    """
    authority_host_uri = 'https://login.microsoftonline.com'
    authority_uri = authority_host_uri + '/' + tenant
    resource_uri = 'https://management.core.windows.net/'
    
    context = adal.AuthenticationContext(authority_uri, api_version=None)
    mgmt_token = context.acquire_token_with_client_credentials(resource_uri, client_id, client_secret)
    credentials = AADTokenCredentials(mgmt_token, client_id)

    return credentials


In [14]:
credentials = authenticate_client_key(tenant, client_id, client_secret)
access_token = credentials.token.get('access_token')

In [15]:
azure_mgmt_uri = 'https://management.azure.com:443/subscriptions/{subscriptionId}'.format(subscriptionId = subscription)

In [16]:
azure_mgmt_uri

'https://management.azure.com:443/subscriptions/3e6b71a1-1c47-4188-a4dc-793259a87549'

In [17]:
# Azure Resource RateCard API 
# https://docs.microsoft.com/en-us/azure/billing/billing-usage-rate-card-overview
uri_str = "{azure_mgmt_uri}/providers/Microsoft.Commerce/RateCard?" + \
    "api-version=2016-08-31-preview&$filter=OfferDurableId eq '{offerId}' and " + \
    "Currency eq '{currencyId}' and Locale eq '{localeId}' and RegionInfo eq '{regionId}'"
    
rateCardUrl = uri_str.format(
    azure_mgmt_uri = azure_mgmt_uri,
    offerId = offer, 
    currencyId = currency, 
    localeId = locale, 
    regionId = region)

# Don't allow redirects and call the RateCard API
response = requests.get(rateCardUrl, allow_redirects=False, headers = {'Authorization': 'Bearer %s' %access_token})

# Look at response headers to get the redirect URL
redirectUrl = response.headers['Location']

# Get the ratecard content by making another call to go the redirect URL
rateCard = requests.get(redirectUrl)

In [18]:
import pandas as pd
import json

In [19]:
r = json.loads(rateCard.content.decode("utf-8"))
df_rates = pd.DataFrame.from_dict(r['Meters'])
df_rates

,EffectiveDate,IncludedQuantity,MeterCategory,MeterId,MeterName,MeterRates,MeterRegion,MeterStatus,MeterSubCategory,MeterTags,Unit
0,2017-04-01T00:00:00Z,0.0,Virtual Machines,d0bf9053-17c4-4fec-8502-4eb8376343a7,F2/F2s Low Priority,{'0': 0.077},US West 2,Active,F/FS Series Windows,[],1 Hour
1,2017-02-01T00:00:00Z,0.0,Data Services,8b7672d4-16fc-446e-9935-cf2223c5290f,Standard S3 Secondary Database Days,{'0': 4.0177},KR South,Active,SQL Database,[],Days
2,2014-05-01T00:00:00Z,0.0,Virtual Machines,2c4db260-4bc7-4388-a42f-b0c709255ae8,A6,{'0': 0.64},AP Southeast,Active,A Series Windows,[],1 Hour
3,2017-10-20T00:00:00Z,0.0,Cloud Services,c9c058cb-822e-46ae-86fb-6b583f91b4b5,Compute Hours,{'0': 4.676},AU East,Active,Standard_H16r Cloud Services Low Priority,[],1 Hour
4,2016-02-01T00:00:00Z,0.0,Cloud Services,0aaf975b-8e33-404b-93f7-de7d69774ca1,Compute Hours,{'0': 0.14},US West,Active,Standard_D1_v2 Cloud Services,[],Hours
5,2018-07-13T00:00:00Z,0.0,Storage,e1e7c9cb-ed36-4f0b-b3f5-c3dead188ef4,Archive Read Additional IO,{'0': 0.039},AU East,Active,Azure Data Lake Storage Gen2 Hierarchical Name...,[],10K Operations
6,2016-02-01T00:00:00Z,0.0,Cloud Services,77860eb6-e2bc-4d06-8efe-491d840adaf5,Compute Hours,{'0': 3.518},AP East,Active,Standard_D15_v2 Cloud Services,[],Hours
7,2017-01-01T00:00:00Z,0.0,Storage,8ebaf4a8-960b-457b-ba6b-70b59c959ec7,Premium Storage - Page Blob/P60 (Units),{'0': 946.08},US West,Active,Locally Redundant,[],Units
8,2016-09-01T00:00:00Z,0.0,SQL Data Warehouse,22e690b0-183e-4266-bbeb-0c8ccf98f160,100 DWU,{'0': 1.5121},US Central,Active,Compute Optimized Gen1,[],Hours
9,2017-11-03T00:00:00Z,0.0,Cloud Services,1c389e44-fc99-46dd-93b4-5aab8f611ccc,Compute Hours,{'0': 3.04},JA West,Active,Standard_E64_v3 Cloud Services Low Priority,[],1 Hour


In [21]:
uri_str = "{azure_mgmt_uri}/providers/Microsoft.Commerce/UsageAggregates?" + \
    "api-version=2015-06-01-preview&" + \
    "aggregationGranularity=Daily&" + \
    "reportedstartTime=2018-06-12+00%3a00%3a00Z&" + \
    "reportedEndTime=2018-06-14+00%3a00%3a00Z"
usage_url = uri_str.format(azure_mgmt_uri = azure_mgmt_uri)
usage_url

'https://management.azure.com:443/subscriptions/3e6b71a1-1c47-4188-a4dc-793259a87549/providers/Microsoft.Commerce/UsageAggregates?api-version=2015-06-01-preview&aggregationGranularity=Daily&reportedstartTime=2018-06-12+00%3a00%3a00Z&reportedEndTime=2018-06-14+00%3a00%3a00Z'

In [22]:
response = requests.get(usage_url, allow_redirects=False, headers = {'Authorization': 'Bearer %s' %access_token})
usage = response.json()

In [26]:
# usage API doesn't return more than 1000 aggregates 
# see https://stackoverflow.com/questions/50948666/get-azurermconsumptionusagedetail-limited-response-to-1000-items
len(usage['value'])

20

In [27]:
df_usage = pd.DataFrame.from_dict(usage['value'])
df_usage

,id,name,properties,type
0,/subscriptions/3e6b71a1-1c47-4188-a4dc-793259a...,Daily_BRSDT_20180612_0000,{'subscriptionId': '3e6b71a1-1c47-4188-a4dc-79...,Microsoft.Commerce/UsageAggregate
1,/subscriptions/3e6b71a1-1c47-4188-a4dc-793259a...,Daily_BRSDT_20180612_0000,{'subscriptionId': '3e6b71a1-1c47-4188-a4dc-79...,Microsoft.Commerce/UsageAggregate
2,/subscriptions/3e6b71a1-1c47-4188-a4dc-793259a...,Daily_BRSDT_20180612_0000,{'subscriptionId': '3e6b71a1-1c47-4188-a4dc-79...,Microsoft.Commerce/UsageAggregate
3,/subscriptions/3e6b71a1-1c47-4188-a4dc-793259a...,Daily_BRSDT_20180612_0000,{'subscriptionId': '3e6b71a1-1c47-4188-a4dc-79...,Microsoft.Commerce/UsageAggregate
4,/subscriptions/3e6b71a1-1c47-4188-a4dc-793259a...,Daily_BRSDT_20180612_0000,{'subscriptionId': '3e6b71a1-1c47-4188-a4dc-79...,Microsoft.Commerce/UsageAggregate
5,/subscriptions/3e6b71a1-1c47-4188-a4dc-793259a...,Daily_BRSDT_20180612_0000,{'subscriptionId': '3e6b71a1-1c47-4188-a4dc-79...,Microsoft.Commerce/UsageAggregate
6,/subscriptions/3e6b71a1-1c47-4188-a4dc-793259a...,Daily_BRSDT_20180612_0000,{'subscriptionId': '3e6b71a1-1c47-4188-a4dc-79...,Microsoft.Commerce/UsageAggregate
7,/subscriptions/3e6b71a1-1c47-4188-a4dc-793259a...,Daily_BRSDT_20180612_0000,{'subscriptionId': '3e6b71a1-1c47-4188-a4dc-79...,Microsoft.Commerce/UsageAggregate
8,/subscriptions/3e6b71a1-1c47-4188-a4dc-793259a...,Daily_BRSDT_20180612_0000,{'subscriptionId': '3e6b71a1-1c47-4188-a4dc-79...,Microsoft.Commerce/UsageAggregate
9,/subscriptions/3e6b71a1-1c47-4188-a4dc-793259a...,Daily_BRSDT_20180612_0000,{'subscriptionId': '3e6b71a1-1c47-4188-a4dc-79...,Microsoft.Commerce/UsageAggregate


In [29]:
# create a dataframe form the 'properties' key on each usage record
df_usage_detail = pd.DataFrame(df_usage['properties'].values.tolist())
df_usage_detail

,infoFields,instanceData,meterCategory,meterId,meterName,meterRegion,meterSubCategory,quantity,subscriptionId,unit,usageEndTime,usageStartTime
0,{},"{""Microsoft.Resources"":{""resourceUri"":""/subscr...",Storage,e9549cbe-02d9-4213-b4be-22d6dfe8a3af,Premium Storage - Page Blob/P10 (Units),US West,Locally Redundant,0.001389,3e6b71a1-1c47-4188-a4dc-793259a87549,Units,2018-06-13T00:00:00+00:00,2018-06-12T00:00:00+00:00
1,{},"{""Microsoft.Resources"":{""resourceUri"":""/subscr...",Data Management,9cb0bde8-bc0d-468c-8423-a25fe06779d3,Standard IO - Table Write Operation Units (in ...,NaN,NaN,0.000500,3e6b71a1-1c47-4188-a4dc-793259a87549,"10,000s",2018-06-13T00:00:00+00:00,2018-06-12T00:00:00+00:00
2,{},"{""Microsoft.Resources"":{""resourceUri"":""/subscr...",Networking,9995d93a-7d35-4d3f-9c69-7a7fea447ef4,Data Transfer Out (GB),Zone 1,NaN,0.059841,3e6b71a1-1c47-4188-a4dc-793259a87549,GB,2018-06-13T00:00:00+00:00,2018-06-12T00:00:00+00:00
3,{},"{""Microsoft.Resources"":{""resourceUri"":""/subscr...",Virtual Machines,d101de3e-ae70-48bb-8605-64fcd0a3ce8f,D4 v3/D4s v3,US West,Dv3/DSv3 Series Windows,0.766682,3e6b71a1-1c47-4188-a4dc-793259a87549,1 Hour,2018-06-13T00:00:00+00:00,2018-06-12T00:00:00+00:00
4,{},"{""Microsoft.Resources"":{""resourceUri"":""/subscr...",Data Management,b9e5e77c-a0b3-4a2c-9b8b-57fa54f31c52,Standard IO - Table Batch Write Operation Unit...,NaN,NaN,0.000100,3e6b71a1-1c47-4188-a4dc-793259a87549,"10,000s",2018-06-13T00:00:00+00:00,2018-06-12T00:00:00+00:00
5,{},"{""Microsoft.Resources"":{""resourceUri"":""/subscr...",Networking,d54686f0-77ff-43f3-9e7c-2099030d32a7,DNS Queries (1M),NaN,DNS,0.000804,3e6b71a1-1c47-4188-a4dc-793259a87549,1M Queries,2018-06-13T00:00:00+00:00,2018-06-12T00:00:00+00:00
6,{},"{""Microsoft.Resources"":{""resourceUri"":""/subscr...",Networking,f114cb19-ea64-40b5-bcd7-aee474b62853,IP Address Hours,NaN,Public IP Addresses,0.900000,3e6b71a1-1c47-4188-a4dc-793259a87549,Hours,2018-06-13T00:00:00+00:00,2018-06-12T00:00:00+00:00
7,{},"{""Microsoft.Resources"":{""resourceUri"":""/subscr...",Data Management,c80a3636-2edb-4248-bcb1-04ef818a75ac,Standard IO - Disk Write Operation Units (in 1...,NaN,NaN,0.009300,3e6b71a1-1c47-4188-a4dc-793259a87549,"10,000s",2018-06-13T00:00:00+00:00,2018-06-12T00:00:00+00:00
8,{},"{""Microsoft.Resources"":{""resourceUri"":""/subscr...",Data Management,923978e1-fd3f-4bd5-a798-f4b533057e46,Standard IO - Block Blob Delete Operation Unit...,NaN,NaN,0.004500,3e6b71a1-1c47-4188-a4dc-793259a87549,"10,000s",2018-06-13T00:00:00+00:00,2018-06-12T00:00:00+00:00
9,{},"{""Microsoft.Resources"":{""resourceUri"":""/subscr...",Networking,32c3ebec-1646-49e3-8127-2cafbd3a04d8,Data Transfer In (GB),Zone 1,NaN,1.619603,3e6b71a1-1c47-4188-a4dc-793259a87549,GB,2018-06-13T00:00:00+00:00,2018-06-12T00:00:00+00:00


In [30]:
# https://msdn.microsoft.com/en-us/library/azure/mt219001.aspx
# Note the query is by *reported* time, but the return is by *usage* time
# "... we ask callers to query by Reported Time to ensure that they get all the usage 
# events reported within a specific time period within the billing system. Even though the query is made 
# with the Reported Time, the usage response is aggregated by the resource usage time, which is the useful 
# pivot for callers."
# We queried from 6-12 to 6-14. This usage shows up on 6-12 to 6-13, but if you quesry
# 6-12 to 6-13 this record will not be picked up as it was *reported* 6-13 to 6-14

df_usage_detail.loc[df_usage_detail['meterId'] == "9995d93a-7d35-4d3f-9c69-7a7fea447ef4"]

,infoFields,instanceData,meterCategory,meterId,meterName,meterRegion,meterSubCategory,quantity,subscriptionId,unit,usageEndTime,usageStartTime
2,{},"{""Microsoft.Resources"":{""resourceUri"":""/subscr...",Networking,9995d93a-7d35-4d3f-9c69-7a7fea447ef4,Data Transfer Out (GB),Zone 1,NaN,0.059841,3e6b71a1-1c47-4188-a4dc-793259a87549,GB,2018-06-13T00:00:00+00:00,2018-06-12T00:00:00+00:00


In [31]:
invoice_list_url = "https://management.azure.com:443/subscriptions/{subscriptionId}/providers/Microsoft.Billing/invoices?api-version=2017-04-24-preview".format(subscriptionId = subscription)
invoice_list_url

'https://management.azure.com:443/subscriptions/3e6b71a1-1c47-4188-a4dc-793259a87549/providers/Microsoft.Billing/invoices?api-version=2017-04-24-preview'

In [32]:
response = requests.get(invoice_list_url, allow_redirects=False, headers = {'Authorization': 'Bearer %s' %access_token})

In [34]:
# There are 3 invoices against this subscription so far
response.json()

{'value': [{'id': '/subscriptions/3e6b71a1-1c47-4188-a4dc-793259a87549/providers/Microsoft.Billing/invoices/201810-318387190656310',
   'type': 'Microsoft.Billing/invoices',
   'name': '201810-318387190656310',
   'properties': {'invoicePeriodEndDate': '2018-09-11',
    'invoicePeriodStartDate': '2018-08-12',
    'billingPeriodIds': ['/subscriptions/3e6b71a1-1c47-4188-a4dc-793259a87549/providers/Microsoft.Billing/billingPeriods/201810-1']}},
  {'id': '/subscriptions/3e6b71a1-1c47-4188-a4dc-793259a87549/providers/Microsoft.Billing/invoices/201809-318692150622116',
   'type': 'Microsoft.Billing/invoices',
   'name': '201809-318692150622116',
   'properties': {'invoicePeriodEndDate': '2018-08-11',
    'invoicePeriodStartDate': '2018-07-12',
    'billingPeriodIds': ['/subscriptions/3e6b71a1-1c47-4188-a4dc-793259a87549/providers/Microsoft.Billing/billingPeriods/201809-1']}},
  {'id': '/subscriptions/3e6b71a1-1c47-4188-a4dc-793259a87549/providers/Microsoft.Billing/invoices/201808-31822215060

In [35]:
invoice_name = response.json()['value'][0]['name']
invoice_name

'201810-318387190656310'

In [36]:
invoice_url = "https://management.azure.com:443/subscriptions/{subscriptionId}/providers/Microsoft.Billing/invoices/{invoiceName}?api-version=2017-04-24-preview".format(subscriptionId = subscription, invoiceName=invoice_name)

In [37]:
# download invoice pdf
response = requests.get(invoice_url, allow_redirects=False, headers = {'Authorization': 'Bearer %s' %access_token})


In [38]:
invoice_pdf_url = response.json()['properties']['downloadUrl']['url']

In [39]:
response = requests.get(invoice_pdf_url)

In [41]:
f = open('invoice.pdf', 'wb')
f.write(response.content)
f.close()

In [42]:
!open invoice.pdf

In [25]:
# pickle everything
import pickle
pickle.dump( df_usage, open( "df_usage.p", "wb" ) )
pickle.dump( df_rates, open( "df_rates.p", "wb" ) )
pickle.dump( df_usage_detail, open( "df_usage_detail.p", "wb" ) )

In [ ]:
# TODO
# aggregate usage on rate
# left join usage on rates
# iter 
#   process row into rated record